<a href="https://colab.research.google.com/github/fmejias/CS534-ArtificialIntelligenceProject/blob/main/AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS 534 - Artificial Intelligence**

## **Project Title: Text Mining and Sentiment Analysis on Twitter for predicting students dropping out during the pandemic.**

### **Students**


*   Merzia Adamjee
*   Alketa Guxha
*   Felipe Mejias
*   Nikita Boguslavskii




# **Initial configuration of the environment for the development of the project**

In [3]:
from google.colab import drive
from google.colab import files
import pandas as pd
import re
import numpy as np

from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

# **Install Textblob and Imbalanced Learn**

In [ ]:
!pip install textblob
!pip install imbalanced-learn
!pip3 install flair

# **Google Authentication to read CSV File from Google Drive**

In [5]:
# Needed for Google Authentication Step
drive.mount('/content/drive')

Mounted at /content/drive


# **Upload Dataset from Google Drive**

In [6]:
DATASET_PATH = "/content/drive/My Drive/AI_Project_CS_534/Datasets/dropping_out_tweets_part1_labeled.csv"
dataset_df = pd.read_csv(DATASET_PATH, sep=";")

# **Dataset Information**

In [ ]:
# Print a summary of the Dataset
result = dataset_df.head(10)
print("First 10 rows of the DataFrame:")
print(result)

# **Dataset Preprocessing**

## **Select labeled dataset**

In [279]:
# NOTE: This selection is because they are the only label rows
labeled_dataset_df = dataset_df.head(1200)

## **Filtering irrelevant examples**

In [280]:
IRRELEVANT_KEYWORDS = ["Bernie", "Trump", "Sanders", "to become", 
                       "to pursue", "and becoming", "and going",
                       "and be", "so I can", "so i can", "to run",
                       "to spend", "to focus", "and living", "marry",
                       "stripper", "and joining", "and pursuing",
                       "bts", "BTS", "and running", "to go", "and making",
                       "to dedicate"]

def filtering_irrelevant_examples(twitter_dataset):
  """
  Filtering irrelevant tweets from the Twitter dataset.
  """
  def check_tweet_relevance(tweet):
    """
    Filtering irrelevant tweets from the Twitter dataset.
    """
    if any(indicator in tweet for indicator in IRRELEVANT_KEYWORDS):
      return "irrelevant"
    return "relevant"
  return twitter_dataset[twitter_dataset["tweet"].apply(check_tweet_relevance) \
                         != "irrelevant"]

# Filter the irrelevant tweets
labeled_dataset_df = filtering_irrelevant_examples(labeled_dataset_df)
rows, columns = labeled_dataset_df.shape
print("New number of rows: ", rows)

New number of rows:  936


## **Convert all letters to lower case**

In [281]:
def convert_letters_to_lower_case(twitter_dataset):
  """
  Convert all letters to lower case.
  """
  def tweet_to_lower_case(tweet):
    """
    Convert tweet text to lower case.
    """
    return tweet.lower()

  twitter_dataset["tweet"] = twitter_dataset["tweet"].apply(tweet_to_lower_case)

# Convert all tweets to lower case
convert_letters_to_lower_case(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after lower case:")
print(result)

First 10 rows of the DataFrame after lower case:
                     id  ...                     label
0   1309219160828895233  ...      Intention of dropout
1   1308809031583236096  ...  Not intention of dropout
2   1308716552229998593  ...  Not intention of dropout
3   1308483739584835585  ...  Not intention of dropout
4   1308351921875345409  ...  Not intention of dropout
5   1307763236062650368  ...  Not intention of dropout
7   1306800980307083267  ...  Not intention of dropout
8   1305869148463992832  ...  Not intention of dropout
9   1305659343971311616  ...  Not intention of dropout
10  1305617685020053512  ...  Not intention of dropout

[10 rows x 3 columns]


## **Remove usernames that appear within a tweet**

In [282]:
def remove_usernames_from_tweets(twitter_dataset):
  """
  Remove all usernames that appear on a tweet.
  """
  def remove_username(tweet):
    """
    Remove username from tweet.
    """
    return re.sub('@[\w]+','', tweet)

  twitter_dataset["tweet"] = twitter_dataset["tweet"].apply(remove_username)

# Remove all usernames that appear in a tweet
remove_usernames_from_tweets(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ...                     label
0   1309219160828895233  ...      Intention of dropout
1   1308809031583236096  ...  Not intention of dropout
2   1308716552229998593  ...  Not intention of dropout
3   1308483739584835585  ...  Not intention of dropout
4   1308351921875345409  ...  Not intention of dropout
5   1307763236062650368  ...  Not intention of dropout
7   1306800980307083267  ...  Not intention of dropout
8   1305869148463992832  ...  Not intention of dropout
9   1305659343971311616  ...  Not intention of dropout
10  1305617685020053512  ...  Not intention of dropout

[10 rows x 3 columns]


## **Remove hashtags that appear within a tweet**

In [283]:
def remove_hashtags_from_tweets(twitter_dataset):
  """
  Remove all hashtags that appear on a tweet.
  """
  def remove_hashtags(tweet):
    """
    Remove hashtags from tweet.
    """
    return tweet.replace("#", "").replace("_", " ")

  twitter_dataset["tweet"] = twitter_dataset["tweet"].apply(remove_hashtags)

# Remove all hashtags that appear in a tweet
remove_hashtags_from_tweets(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ...                     label
0   1309219160828895233  ...      Intention of dropout
1   1308809031583236096  ...  Not intention of dropout
2   1308716552229998593  ...  Not intention of dropout
3   1308483739584835585  ...  Not intention of dropout
4   1308351921875345409  ...  Not intention of dropout
5   1307763236062650368  ...  Not intention of dropout
7   1306800980307083267  ...  Not intention of dropout
8   1305869148463992832  ...  Not intention of dropout
9   1305659343971311616  ...  Not intention of dropout
10  1305617685020053512  ...  Not intention of dropout

[10 rows x 3 columns]


## **Remove special characters and punctuation that appear within a tweet**

In [284]:
def remove_special_characters_and_punctuation_from_tweets(twitter_dataset):
  """
  Remove all special characters and punctuation that appear on a tweet.
  """
  def remove_special_characters_and_punctuation(tweet):
    """
    Remove special characters and punctuation from tweet.
    """
    return re.sub('[^A-Za-z0-9 ]+', '', tweet)

  twitter_dataset["tweet"] = twitter_dataset["tweet"].apply(remove_special_characters_and_punctuation)

# Remove all special characters and punctuation that appear in a tweet
remove_special_characters_and_punctuation_from_tweets(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ...                     label
0   1309219160828895233  ...      Intention of dropout
1   1308809031583236096  ...  Not intention of dropout
2   1308716552229998593  ...  Not intention of dropout
3   1308483739584835585  ...  Not intention of dropout
4   1308351921875345409  ...  Not intention of dropout
5   1307763236062650368  ...  Not intention of dropout
7   1306800980307083267  ...  Not intention of dropout
8   1305869148463992832  ...  Not intention of dropout
9   1305659343971311616  ...  Not intention of dropout
10  1305617685020053512  ...  Not intention of dropout

[10 rows x 3 columns]


## **Remove URLs that appear within a tweet**

In [285]:
def remove_urls_from_tweets(twitter_dataset):
  """
  Remove all urls that appear on a tweet.
  """
  def remove_urls(tweet):
    """
    Remove urls from tweet.
    """
    return re.sub(r'http\S+', '', tweet)

  twitter_dataset["tweet"] = twitter_dataset["tweet"].apply(remove_urls)

# Remove all urls that appear in a tweet
remove_urls_from_tweets(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ...                     label
0   1309219160828895233  ...      Intention of dropout
1   1308809031583236096  ...  Not intention of dropout
2   1308716552229998593  ...  Not intention of dropout
3   1308483739584835585  ...  Not intention of dropout
4   1308351921875345409  ...  Not intention of dropout
5   1307763236062650368  ...  Not intention of dropout
7   1306800980307083267  ...  Not intention of dropout
8   1305869148463992832  ...  Not intention of dropout
9   1305659343971311616  ...  Not intention of dropout
10  1305617685020053512  ...  Not intention of dropout

[10 rows x 3 columns]


# **Approach using Sentiment Analysis**

## **Create features using sentiment analysis and unigrams and Textblob**

In [286]:
from textblob import TextBlob

def calculate_features_using_polarity(twitter_dataset):
  """
  Use Textblob polarity to calculate the number of positive and negative words.
  """
  def calculate_positive_words(tweet):
    """
    Count number of positive words in a tweet.
    """
    number_of_positive_words = 0
    for word in tweet.split():
      if TextBlob(word).polarity >= 0:
        number_of_positive_words = number_of_positive_words + 1
    return number_of_positive_words
  
  def calculate_ratio_positive_words(tweet):
    """
    Calculate the ratio of positive words in a tweet.
    """
    number_of_positive_words = 0
    for word in tweet.split():
      if TextBlob(word).polarity > 0:
        number_of_positive_words = number_of_positive_words + 1
    return number_of_positive_words/len(tweet.split())
  
  def calculate_negative_words(tweet):
    """
    Count number of negative words in a tweet.
    """
    number_of_negative_words = 0
    for word in tweet.split():
      if TextBlob(word).polarity < 0:
        number_of_negative_words = number_of_negative_words + 1
    return number_of_negative_words
  
  def calculate_ratio_negative_words(tweet):
    """
    Calculate the ratio of negative words in a tweet.
    """
    number_of_negative_words = 0
    for word in tweet.split():
      if TextBlob(word).polarity < 0:
        number_of_negative_words = number_of_negative_words + 1
    return number_of_negative_words/len(tweet.split())
  
  twitter_dataset["unigram_number_positive_words"] = twitter_dataset["tweet"].apply(calculate_positive_words)
  twitter_dataset["unigram_ratio_positive_words"] = twitter_dataset["tweet"].apply(calculate_ratio_positive_words)
  twitter_dataset["unigram_number_negative_words"] = twitter_dataset["tweet"].apply(calculate_negative_words)
  twitter_dataset["unigram_ratio_negative_words"] = twitter_dataset["tweet"].apply(calculate_ratio_negative_words)

# Calculate new features using sentiment Analysis
calculate_features_using_polarity(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ... unigram_ratio_negative_words
0   1309219160828895233  ...                     0.043478
1   1308809031583236096  ...                     0.020833
2   1308716552229998593  ...                     0.046512
3   1308483739584835585  ...                     0.000000
4   1308351921875345409  ...                     0.034483
5   1307763236062650368  ...                     0.000000
7   1306800980307083267  ...                     0.000000
8   1305869148463992832  ...                     0.000000
9   1305659343971311616  ...                     0.000000
10  1305617685020053512  ...                     0.028571

[10 rows x 7 columns]


## **Create features using sentiment analysis and bigrams and Textblob**

In [287]:
from textblob import TextBlob

def find_ngrams(n, input_sequence):
  # Split sentence into tokens.
  tokens = input_sequence.split()
  ngrams = []
  for i in range(len(tokens) - n + 1):
    # Take n consecutive tokens in array.
    ngram = tokens[i:i+n]
    # Concatenate array items into string.
    ngram = ' '.join(ngram)
    ngrams.append(ngram)
  return ngrams

def calculate_bigram_features_using_polarity(twitter_dataset):
  """
  Use Textblob polarity to calculate the number of positive and negative words.
  """
  def calculate_positive_words(tweet):
    """
    Count number of positive words in a tweet.
    """
    number_of_positive_bigrams = 0
    ngrams = find_ngrams(2, tweet)
    for ngram in ngrams:
      if TextBlob(ngram).polarity >= 0:
        number_of_positive_bigrams = number_of_positive_bigrams + 1
    return number_of_positive_bigrams
  
  def calculate_ratio_positive_words(tweet):
    """
    Calculate the ratio of positive words in a tweet.
    """
    number_of_positive_bigrams = 0
    ngrams = find_ngrams(2, tweet)
    for ngram in ngrams:
      if TextBlob(ngram).polarity >= 0:
        number_of_positive_bigrams = number_of_positive_bigrams + 1
    return number_of_positive_bigrams/len(ngrams)
  
  def calculate_negative_words(tweet):
    """
    Count number of negative words in a tweet.
    """
    number_of_negative_bigrams = 0
    ngrams = find_ngrams(2, tweet)
    for ngram in ngrams:
      if TextBlob(ngram).polarity < 0:
        number_of_negative_bigrams = number_of_negative_bigrams + 1
    return number_of_negative_bigrams
  
  def calculate_ratio_negative_words(tweet):
    """
    Calculate the ratio of negative words in a tweet.
    """
    number_of_negative_bigrams = 0
    ngrams = find_ngrams(2, tweet)
    for ngram in ngrams:
      if TextBlob(ngram).polarity < 0:
        number_of_negative_bigrams = number_of_negative_bigrams + 1
    return number_of_negative_bigrams/len(ngrams)
  
  twitter_dataset["bigram_number_positive_words"] = twitter_dataset["tweet"].apply(calculate_positive_words)
  twitter_dataset["bigram_ratio_positive_words"] = twitter_dataset["tweet"].apply(calculate_ratio_positive_words)
  twitter_dataset["bigram_number_negative_words"] = twitter_dataset["tweet"].apply(calculate_negative_words)
  twitter_dataset["bigram_ratio_negative_words"] = twitter_dataset["tweet"].apply(calculate_ratio_negative_words)

# Calculate new features using sentiment Analysis
calculate_bigram_features_using_polarity(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ... bigram_ratio_negative_words
0   1309219160828895233  ...                    0.088889
1   1308809031583236096  ...                    0.042553
2   1308716552229998593  ...                    0.095238
3   1308483739584835585  ...                    0.000000
4   1308351921875345409  ...                    0.071429
5   1307763236062650368  ...                    0.000000
7   1306800980307083267  ...                    0.000000
8   1305869148463992832  ...                    0.000000
9   1305659343971311616  ...                    0.000000
10  1305617685020053512  ...                    0.058824

[10 rows x 11 columns]


## **Create features using sentiment analysis and unigrams and Vader**

In [288]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sentiment_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [289]:
def calculate_features_using_polarity_vader(twitter_dataset):
  """
  Use Vader polarity to calculate the number of positive and negative words.
  """
  def calculate_positive_words(tweet):
    """
    Count number of positive words in a tweet.
    """
    number_of_positive_words = 0
    for word in tweet.split():
      if sentiment_analyzer.polarity_scores(word)["compound"] > 0:
        number_of_positive_words = number_of_positive_words + 1
    return number_of_positive_words
  
  def calculate_ratio_positive_words(tweet):
    """
    Calculate the ratio of positive words in a tweet.
    """
    number_of_positive_words = 0
    for word in tweet.split():
      if sentiment_analyzer.polarity_scores(word)["compound"] > 0:
        number_of_positive_words = number_of_positive_words + 1
    return number_of_positive_words/len(tweet.split())
  
  def calculate_negative_words(tweet):
    """
    Count number of negative words in a tweet.
    """
    number_of_negative_words = 0
    for word in tweet.split():
      if sentiment_analyzer.polarity_scores(word)["compound"] < 0:
        number_of_negative_words = number_of_negative_words + 1
    return number_of_negative_words
  
  def calculate_ratio_negative_words(tweet):
    """
    Calculate the ratio of negative words in a tweet.
    """
    number_of_negative_words = 0
    for word in tweet.split():
      if sentiment_analyzer.polarity_scores(word)["compound"] < 0:
        number_of_negative_words = number_of_negative_words + 1
    return number_of_negative_words/len(tweet.split())
  
  twitter_dataset["unigram_vader_positive_words"] = twitter_dataset["tweet"].apply(calculate_positive_words)
  twitter_dataset["unigram_vader_ratio_positive_words"] = twitter_dataset["tweet"].apply(calculate_ratio_positive_words)
  twitter_dataset["unigram_vader_negative_words"] = twitter_dataset["tweet"].apply(calculate_negative_words)
  twitter_dataset["unigram_vader_ratio_negative_words"] = twitter_dataset["tweet"].apply(calculate_ratio_negative_words)

# Calculate new features using sentiment Analysis
calculate_features_using_polarity_vader(labeled_dataset_df)

# Show results
result = labeled_dataset_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
                     id  ... unigram_vader_ratio_negative_words
0   1309219160828895233  ...                           0.043478
1   1308809031583236096  ...                           0.041667
2   1308716552229998593  ...                           0.093023
3   1308483739584835585  ...                           0.066667
4   1308351921875345409  ...                           0.068966
5   1307763236062650368  ...                           0.023256
7   1306800980307083267  ...                           0.000000
8   1305869148463992832  ...                           0.028571
9   1305659343971311616  ...                           0.000000
10  1305617685020053512  ...                           0.057143

[10 rows x 15 columns]


## **Normalize calculated features**

In [290]:
# Select calculated features
dataset_sentiment_features = labeled_dataset_df[["unigram_number_positive_words", 
                                                 "unigram_ratio_positive_words", 
                                                 "unigram_number_negative_words", 
                                                 "unigram_ratio_negative_words",
                                                 "bigram_number_positive_words", 
                                                 "bigram_ratio_positive_words", 
                                                 "bigram_number_negative_words", 
                                                 "bigram_ratio_negative_words",
                                                 "unigram_vader_positive_words", 
                                                 "unigram_vader_ratio_positive_words", 
                                                 "unigram_vader_negative_words", 
                                                 "unigram_vader_ratio_negative_words"]]

# Normalize using the mean value
normalized_df = (dataset_sentiment_features - dataset_sentiment_features.mean())/dataset_sentiment_features.std()

# Show results
result = normalized_df.head(10)
print("First 10 rows of the DataFrame after removing usernames:")
print(result)

First 10 rows of the DataFrame after removing usernames:
    unigram_number_positive_words  ...  unigram_vader_ratio_negative_words
0                        1.124937  ...                            0.185303
1                        1.351068  ...                            0.138183
2                        0.898806  ...                            1.473973
3                       -1.060996  ...                            0.788436
4                       -0.081095  ...                            0.848229
5                        1.049560  ...                           -0.340684
7                        0.521921  ...                           -0.945570
8                        0.446544  ...                           -0.202425
9                       -0.382603  ...                           -0.945570
10                       0.371167  ...                            0.540721

[10 rows x 12 columns]


## **Handle imbalance classes using SMOTE**

In [ ]:
print("Number of rows with intention of dropout: ", 
      len(labeled_dataset_df[(labeled_dataset_df['label'] == "Intention of dropout")]))
print("Number of rows with no intention of dropout: ", 
      len(labeled_dataset_df[(labeled_dataset_df['label'] == "Not intention of dropout")]))

# Select the data
X = normalized_df
Y = labeled_dataset_df.label

# Transform the dataset
oversample = SMOTE()
X, Y = oversample.fit_resample(X, Y)

## **Train logistic regression model**

In [427]:
# Train the model
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size = 0.2)
logistic_classifier = LogisticRegression(random_state = 0).fit(X_train,
                                                               Y_train)

# Calculate training accuracy
Y_pred = logistic_classifier.predict(X_train)

# Calculate the accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(Y_train, Y_pred))

Accuracy:  0.6949846468781986


## **Train Random Forest Classifier model**

In [422]:
from sklearn.ensemble import RandomForestClassifier

# Train the model
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size = 0.3)
random_forest_classifier = RandomForestClassifier(max_depth=2, random_state=0)
random_forest_classifier.fit(X_train, Y_train)

# Calculate training accuracy
Y_pred = random_forest_classifier.predict(X_train)

# Calculate the accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(Y_train, Y_pred))

Accuracy:  0.7169590643274854


## **Train SVM model**

In [407]:
from sklearn.svm import SVC

# Train the model
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size = 0.3)
svm_classifier = SVC()
svm_classifier.fit(X_train, Y_train)

# Calculate training accuracy
Y_pred = svm_classifier.predict(X_train)

# Calculate the accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(Y_train, Y_pred))

Accuracy:  0.7625730994152047
